In [1]:
import wave
import pyroomacoustics as pra
import numpy as np
from pydub import AudioSegment
import scipy.signal as signal
import scipy as scipy
import os
import matplotlib.pyplot as plt

import sys
sys.path.append("../")
from src.file_io import write_signal_to_wav, write_signal_to_npz
from src.audio_processing import normalize_and_pad_audio_files, calculate_snr

In [2]:
np.random.seed(0)
wave_files = ["../data/raw/samaple/arctic_a0001.wav", "../data/raw/samaple/arctic_a0002.wav"]
audio_data = normalize_and_pad_audio_files(wave_files)

# パラメータの設定
n_sim_sources = 2
sample_rate = 16000
N = 1024
Nk = int(N / 2 + 1)
freqs = np.arange(0, Nk, 1) * sample_rate / N
SNR = 90.0
azimuth_th = 30.0
room_dim = np.array([10.0, 10.0, 10.0])

In [4]:
mic_array_loc = room_dim / 2 + np.random.randn(3) * 0.1
mic_directions = np.array([[np.pi / 2.0, theta / 180.0 * np.pi] for theta in np.arange(180, 361, 180)])

distance = 0.01
mic_alignments = np.zeros((3, mic_directions.shape[0]), dtype=mic_directions.dtype)
mic_alignments[0, :] = np.cos(mic_directions[:, 1]) * np.sin(mic_directions[:, 0])
mic_alignments[1, :] = np.sin(mic_directions[:, 1]) * np.sin(mic_directions[:, 0])
mic_alignments[2, :] = np.array([5, 5])
mic_alignments *= distance

R = mic_alignments + mic_array_loc[:, None]
room = pra.ShoeBox(room_dim, fs=sample_rate, max_order=0)
room.add_microphone_array(pra.MicrophoneArray(R, fs=room.fs))
room_no_noise_left = pra.ShoeBox(room_dim, fs=sample_rate, max_order=0)
room_no_noise_left.add_microphone_array(pra.MicrophoneArray(R, fs=room.fs))
room_no_noise_right = pra.ShoeBox(room_dim, fs=sample_rate, max_order=0)
room_no_noise_right.add_microphone_array(pra.MicrophoneArray(R, fs=room.fs))

# 場所
doas = np.array([[np.pi / 2.0, np.pi], [np.pi / 2.0, 0]])
# 音源とマイクロホンの距離
distance = 1.0

source_locations = np.zeros((3, doas.shape[0]), dtype=doas.dtype)
source_locations[0, :] = np.cos(doas[:, 1]) * np.sin(doas[:, 0])
source_locations[1, :] = np.sin(doas[:, 1]) * np.sin(doas[:, 0])
# source_locations[2, :] = np.cos(doas[:, 0])
source_locations *= distance
source_locations += mic_array_loc[:, None]

In [5]:
# シミュレーションを回す
for s in range(n_sim_sources):
    room.add_source(source_locations[:, s], signal=audio_data[s])
    if s == 0:
        room_no_noise_left.add_source(source_locations[:, s], signal=audio_data[s])
    if s == 1:
        room_no_noise_right.add_source(source_locations[:, s], signal=audio_data[s])

room.simulate(snr=SNR)
room_no_noise_left.simulate(snr=90)
room_no_noise_right.simulate(snr=90)
multi_conv_data = room.mic_array.signals
multi_conv_data_left_no_noise = room_no_noise_left.mic_array.signals
multi_conv_data_right_no_noise = room_no_noise_right.mic_array.signals

In [7]:
# 短時間フーリエ変換を行う
f, t, stft_data = signal.stft(multi_conv_data, fs=sample_rate, window="hann", nperseg=N)

# ICAの繰り返し回数
n_ica_iterations = 50

# Pyroomacousticsによる音源分離
# nframes, nfrequencies, nchannels
# 入力信号のインデックスの順番を( M, Nk, Lt)から(Lt,Nk,M)に変換する
y_pa_auxiva = pra.bss.auxiva(np.transpose(stft_data, (2, 1, 0)), n_iter=n_ica_iterations)
y_pa_auxiva = np.transpose(y_pa_auxiva, (2, 1, 0))[None, ...]

y_pa_ilrma = pra.bss.ilrma(np.transpose(stft_data, (2, 1, 0)), n_iter=n_ica_iterations)
y_pa_ilrma = np.transpose(y_pa_ilrma, (2, 1, 0))[None, ...]

t, y_pa_auxiva = signal.istft(y_pa_auxiva[0, ...], fs=sample_rate, window="hann", nperseg=N)
t, y_pa_ilrma = signal.istft(y_pa_ilrma[0, ...], fs=sample_rate, window="hann", nperseg=N)

snr_pre = calculate_snr(multi_conv_data_left_no_noise[0, ...], multi_conv_data[0, ...]) + calculate_snr(
    multi_conv_data_right_no_noise[0, ...], multi_conv_data[0, ...]
)
snr_pre /= 2.0

snr_pa_ilrma_post1 = calculate_snr(multi_conv_data_left_no_noise[0, ...], y_pa_ilrma[0, ...]) + calculate_snr(
    multi_conv_data_right_no_noise[0, ...], y_pa_ilrma[1, ...]
)
snr_pa_ilrma_post2 = calculate_snr(multi_conv_data_left_no_noise[0, ...], y_pa_ilrma[1, ...]) + calculate_snr(
    multi_conv_data_right_no_noise[0, ...], y_pa_ilrma[0, ...]
)

snr_pa_ilrma_post = np.maximum(snr_pa_ilrma_post1, snr_pa_ilrma_post2)
snr_pa_ilrma_post /= 2.0

print(
    "Δsnr [dB]: {:.2f}   {:.2f}".format(
        snr_pa_ilrma_post - snr_pre,
        snr_pa_ilrma_post - snr_pre,
    )
)

Δsnr [dB]: 15.57   15.57


In [7]:
write_signal_to_wav(multi_conv_data_left_no_noise, "./no_noise_left.wav", sample_rate)
write_signal_to_wav(multi_conv_data_right_no_noise, "./no_noise_right.wav", sample_rate)
write_signal_to_wav(multi_conv_data, "./multi_conv_data.wav", sample_rate)

write_signal_to_wav(y_pa_auxiva, "./pa_auxiva.wav", sample_rate)
write_signal_to_wav(y_pa_ilrma, "./pa_ilrma.wav", sample_rate)

TODO: スペクトログラム表示のための関数を作成